# Air Canvas

# Importing Libraries

In [2]:
import cv2 as cv
import numpy as np
from collections import deque

# Defining Color

In [3]:
def Colors():
    R=(255,0,0)
    B=(0,255,0)
    G=(0,0,255)
    Y=(0,255,255)
    return [R,B,G,Y]

# Defining Range For pointer Hue Saturation Value

In [14]:
def SettingPointerRange():
    UpperHue=130
    UpperValue=255
    UpperSaturation=255
    #64,72,49
    LowerHue=110
    LowerSaturation=50
    LowerValue=50
    UpperRange=np.array([UpperHue,UpperSaturation,UpperValue])
    LowerRange=np.array([LowerHue,LowerSaturation,LowerValue])
    return UpperRange,LowerRange

# Setting the text on Window

In [15]:
def SetTextToWindow(window,NameOfString,CoordinatesOfBottomLeft,ColorOfText):
    lineType=cv.LINE_AA
    fontScale=0.6
    ThicknessOfLineInPixel=2
    cv.putText(window, NameOfString, CoordinatesOfBottomLeft, cv.FONT_HERSHEY_SIMPLEX, fontScale, ColorOfText, ThicknessOfLineInPixel,lineType )
    

# Data Structure To Keep the track of Pixels

In [16]:
def MakeStructure(size):
    return [deque(maxlen=size)],[deque(maxlen=size)],[deque(maxlen=size)],[deque(maxlen=size)]
    

In [17]:
def SetColoredArea(WindowName,StartingPosition,EndingPosition,Color,x):
    return cv.rectangle(WindowName,StartingPosition,EndingPosition,Color,x)

# Creating Canvas Window

In [18]:
def CanvasWindow():
    RGBColor=Colors()
    #creating canvas window color white of whole window
    Canvas= np.zeros((471,636,3)) + 255
    #setting the colors for clear rectangle on window
   # Canvas = SetColoredArea(Canvas, (40,1), (140,65), (0,0,0), 2)
    #setting the color for red ractangle on window
   # Canvas = SetColoredArea(Canvas, (160,1), (255,65), RGBColor[0], -1)
    #setting the color for blue ractangle on window
   # Canvas = SetColoredArea(Canvas, (275,1), (370,65), RGBColor[1], -1)
    #setting the color for green ractangle on window
    #Canvas = SetColoredArea(Canvas, (390,1), (485,65), RGBColor[2], -1)
    #setting the color for yellow ractangle on window
    #Canvas = SetColoredArea(Canvas, (505,1), (600,65), RGBColor[3], -1)
    
    
    return Canvas
    

In [19]:
# Holding the pixels location of different colors
TrackBluePoints,TrackRedPoints,TrackGreenPoints,TrackYellowPoints=MakeStructure(2048)

In [20]:
# These indexes will be used to mark the points in particular arrays of specific colour
blue_index = 0
green_index = 0
red_index = 0
yellow_index = 0

In [21]:
#getting the colors rgb 
colors = Colors()
colorIndex = 0

# Drawing Lines by iterating each location in respective data Structure

In [22]:
def FillRespectivePixels(a):
    for i in range(len(a)):
        for j in range(len(a[i])):
            for k in range(1, len(a[i][j])):
                if a[i][j][k - 1] is None or a[i][j][k] is None:
                    continue
                cv.line(Currentframe, a[i][j][k - 1], a[i][j][k], colors[i], 2)
                cv.line(Canvas, a[i][j][k - 1], a[i][j][k], colors[i], 2)

In [23]:



#The kernel to be used for dilation purpose 
kernel = np.ones((5,5),np.uint8)



# Here is code for Canvas setup
Canvas=CanvasWindow()
cv.namedWindow('Canvas', cv.WINDOW_AUTOSIZE)




# Loading the default webcam of PC.
cap = cv.VideoCapture(0)


while True:
    # Reading the frame from the camera
    ret, Currentframe = cap.read()
    #Flipping the frame to see same side of yours
    Currentframe = cv.flip(Currentframe, 1)
    
    #this function will return the image in hsv form
    hsv = cv.cvtColor(Currentframe, cv.COLOR_BGR2HSV)

    #setting the pointer hue saturation and value range
    Upper_hsv,Lower_hsv=SettingPointerRange()
    
    
   

    # adding the color buttons to the camera frame 
    Currentframe = SetColoredArea(Currentframe, (40,1), (140,65), (122,122,122), 2)
    Currentframe = SetColoredArea(Currentframe, (160,1), (255,65), colors[0], -1)
    Currentframe = SetColoredArea(Currentframe, (275,1), (370,65), colors[1], -1)
    Currentframe = SetColoredArea(Currentframe, (390,1), (485,65), colors[2], -1)
    Currentframe = SetColoredArea(Currentframe, (505,1), (600,65), colors[3], -1)
    #Now do the adding text to the respective colored region in camera frame
    
    SetTextToWindow(Currentframe, "Clear", (49, 33), (255, 255, 255))
    SetTextToWindow(Currentframe, "Blue", (185, 33), (255, 255, 255))
    SetTextToWindow(Currentframe, "Green", (298, 33), (255, 255, 255))
    SetTextToWindow(Currentframe, "Red", (420, 33), (255, 255, 255))
    SetTextToWindow(Currentframe, "Yellow", (520, 33), (150,150,150))


    # Identifying the pointer 
    #it will return the array consisting of 255 and 0 as value in pixel location thresholded image 0 as placed where color lies
    
    ThresholdedImage = cv.inRange(hsv, Lower_hsv, Upper_hsv)
    
    #here morphological operations are applied to remove imperfections(some dots on thresholded image)
    #erosion shrinks forground and enlarge background(Structuring Element of fit means all pixels should match)
    #dilation shrinks background and enlarge foreground(Structuring Elements fit or touch means atleast one pixel match)
    
    
    
    ErodedThresholdedImage = cv.erode(ThresholdedImage, kernel, iterations=1)
    #Now all the imperfections from binary image has gone 
    #
    ClearedErodedThresholdedImage = cv.morphologyEx(ErodedThresholdedImage, cv.MORPH_OPEN, kernel)
    #may there is chance that white shape is shrinked too much so dialating now
    
    DilatedImage = cv.dilate(ClearedErodedThresholdedImage, kernel, iterations=1)

    # Find contours for the pointer after idetifying it
    cnts,_ = cv.findContours(DilatedImage.copy(), cv.RETR_EXTERNAL,
    	cv.CHAIN_APPROX_SIMPLE)
    center = None

    # Ifthe contours are formed
    if len(cnts) > 0:
    	# sorting the contours to find biggest 
        cnt = sorted(cnts, key = cv.contourArea, reverse = True)[0]
        # Get the radius of the enclosing circle around the found contour
        ((x, y), radius) = cv.minEnclosingCircle(cnt)
        # Draw the circle around the contour
        cv.circle(Currentframe, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        # Calculating the center of the detected contour
        M = cv.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

        # Now checking if the user wants to click on any button above the screen 
        if center[1] <= 65:
            if 40 <= center[0] <= 140: # Clear Button
                TrackBluePoints,TrackRedPoints,TrackGreenPoints,TrackYellowPoints=MakeStructure(512)

                blue_index = 0
                green_index = 0
                red_index = 0
                yellow_index = 0

                Canvas[67:,:,:] = 255
            elif 160 <= center[0] <= 255:
                    colorIndex = 0 # Blue
            elif 275 <= center[0] <= 370:
                    colorIndex = 1 # Green
            elif 390 <= center[0] <= 485:
                    colorIndex = 2 # Red
            elif 505 <= center[0] <= 600:
                    colorIndex = 3 # Yellow
        else :
            if colorIndex == 0:
                TrackBluePoints[blue_index].appendleft(center)
            elif colorIndex == 1:
                TrackGreenPoints[green_index].appendleft(center)
            elif colorIndex == 2:
                TrackRedPoints[red_index].appendleft(center)
            elif colorIndex == 3:
                TrackYellowPoints[yellow_index].appendleft(center)
    # Append the next deques when nothing is detected to avois messing up
    else:
        TrackBluePoints.append(deque(maxlen=512))
        blue_index += 1
        TrackGreenPoints.append(deque(maxlen=512))
        green_index += 1
        TrackRedPoints.append(deque(maxlen=512))
        red_index += 1
        TrackYellowPoints.append(deque(maxlen=512))
        yellow_index += 1

    # Draw lines of all the colors on the canvas and frame 
    points = [TrackBluePoints, TrackGreenPoints, TrackRedPoints, TrackYellowPoints]
    FillRespectivePixels(points)

    # Show all the windows
    cv.imshow("Tracking", Currentframe)
    cv.imshow("Canvas", Canvas)
    cv.imshow("mask",DilatedImage)

	# If the 'q' key is pressed then stop the application 
    if cv.waitKey(1) & 0xFF == ord("q"):
        break

# Release the camera and all resources
cap.release()
cv.destroyAllWindows()